# 利用多层全连接神经网络及简单卷积神经网络对MNIST数据集分类

导入相关包

In [1]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn

定义是否使用GPU

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

定义多层全连接神经网络

In [3]:
class multlayer_network(nn.Module):
    '''
        全连接神经网络
    '''
    def __init__(self):
        # 调用父类的构造函数
        super(multlayer_network, self).__init__()
        
        # 构造全连接神经网络
        self.net = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )
        
    def forward(self, tmp):
        return self.net(tmp)

定义LeNet神经网络

In [4]:
class LeNet(nn.Module):
    '''
        LeNet神经网络
    '''
    def __init__(self):
        # 调用父类的构造函数
        super(LeNet, self).__init__()
        
        # 卷积层块
        self.cp1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), # 卷积层,保持形状不变
            nn.ReLU(), # 激活函数
            nn.MaxPool2d(kernel_size=2, stride=2) # 池化层,形状变为(6*14*14)
        )
        self.cp2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5), # 卷积层,形状变为(16*10*10)
            nn.ReLU(), # 激活函数
            nn.MaxPool2d(kernel_size=2, stride=2) # 池化层,形状变为(16*5*5)
        )
        
        # 全连接层块
        self.mult = multlayer_network()
    
    def forward(self, tmp):
        # 卷积层块
        tmp = self.cp1(tmp)
        tmp = self.cp2(tmp)
        
        # 全连接层块
        tmp = tmp.view(tmp.size(0), -1)
        tmp = self.mult(tmp)
        return tmp

超参数设置

In [5]:
EPOCH = 10  # 遍历数据集次数
BATCH_SIZE = 64  # 批处理尺寸(batch_size)
LR = 0.01  # 学习率

定义数据预处理方式

In [6]:
transform = transforms.ToTensor()
trainset = tv.datasets.MNIST(
    root='/data/',
    train=True,
    download=True,
    transform=transform)

# 定义训练批处理数据
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    )

# 定义测试数据集
testset = tv.datasets.MNIST(
    root='/data/',
    train=False,
    download=True,
    transform=transform)

# 定义测试批处理数据
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    )

实例化网络，如有GPU，将模型加载至GPU运算

In [7]:
net = LeNet().to(device)

定义损失函数和优化方式，尝试SGD、Adam等优化器，尝试运用权重衰减

In [8]:
# 使用交叉熵损失函数
criterion = nn.CrossEntropyLoss()

# 设置优化器
optimizer = torch.optim.SGD(net.parameters(), lr=LR)

模型训练及测试

In [9]:
for epoch in range(EPOCH):
    '''
        完成训练过程。如有GPU，将数据加载至GPU运算
    '''
    sum_loss = 0
    # 数据读取
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前馈并反向传播更新网络参数
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()

    print('第{0}轮 loss: {1:.4f}'.format(epoch+1, sum_loss/100))
    
    net.eval() # 进入评估模式
    
    # 每跑完一次epoch测试一下准确率
    with torch.no_grad():
        '''
           完成测试过程，输出最优测试集准确率
        '''
        total = 0
        right_pred = 0
        
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            right_pred += (pred == labels).sum() 
            acc = 100 * right_pred / total
        print('第{0}轮的识别准确率为：{1:.2f}%'.format(epoch+1, acc))

第1轮 loss: 17.8064
第1轮的识别准确率为：77.52%
第2轮 loss: 3.4724
第2轮的识别准确率为：91.66%
第3轮 loss: 2.0347
第3轮的识别准确率为：94.49%
第4轮 loss: 1.4862
第4轮的识别准确率为：96.14%
第5轮 loss: 1.1874
第5轮的识别准确率为：96.63%
第6轮 loss: 0.9977
第6轮的识别准确率为：97.35%
第7轮 loss: 0.8735
第7轮的识别准确率为：97.59%
第8轮 loss: 0.7771
第8轮的识别准确率为：97.24%
第9轮 loss: 0.7101
第9轮的识别准确率为：98.02%
第10轮 loss: 0.6472
第10轮的识别准确率为：97.73%
